In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "10"

from datasets import load_dataset
from transformers import (
    TFWhisperForConditionalGeneration,
    WhisperProcessor,
    WhisperForConditionalGeneration,
)
import torch
from evaluate import load
from itertools import islice
from tqdm import tqdm
from time import time
from collections import defaultdict
import itertools
from pydub import AudioSegment
import numpy as np

/raid/nanosemantics/vyaznikov/itmo_hw/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-23 10:24:53.368549: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 10:24:53.368627: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 10:24:53.368673: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-23 10:24:53.384288: I tensorflow/core/platform/cpu_feature_guard.cc:182] This 

In [2]:
HF_CACHE = "."
NUM_EXAMPLES = 10

In [6]:
def map_to_pred(batch, model, processor):
    audio = batch["audio"]
    audio = AudioSegment(audio["bytes"], frame_rate=16000, sample_width=2, channels=1)
    audio = np.array(audio.get_array_of_samples(), dtype=np.int16) / 32768

    input_features = processor(
        audio, sampling_rate=16000, return_tensors="tf"
    ).input_features
    forced_decoder_ids = processor.get_decoder_prompt_ids(
        language="russian", task="transcribe"
    )

    predicted_ids = model.generate(
        input_features, forced_decoder_ids=forced_decoder_ids
    )[0]
    transcription = processor.decode(predicted_ids)
    return processor.tokenizer._normalize(transcription)


def evaluate(processor, model):
    validation_dataset = load_dataset(
        "bond005/sberdevices_golos_10h_crowd",
        "clean",
        split="validation",
        cache_dir="/raid/nanosemantics/vyaznikov/itmo_hw",
    )
    validation_dataset_slice = islice(validation_dataset, NUM_EXAMPLES)

    res = defaultdict(list)

    t = time()
    for el in tqdm(validation_dataset_slice):
        res["reference"].append(processor.tokenizer._normalize(el["transcription"]))
        res["prediction"].append(map_to_pred(el, model, processor))

    t = ((time() - t) / NUM_EXAMPLES) * 1000
    print(f"Teacher: avg. time on example: {t}")

    wer = load("wer")
    print(
        f"WER on model = {100 * wer.compute(references=res['reference'], predictions=res['prediction'])}"
    )

In [7]:
processor = WhisperProcessor.from_pretrained("openai/whisper-base", cache_dir=HF_CACHE)
model = TFWhisperForConditionalGeneration.from_pretrained(
    "openai/whisper-base", cache_dir=HF_CACHE
)

evaluate(processor, model)

All PyTorch model weights were used when initializing TFWhisperForConditionalGeneration.

All the weights of TFWhisperForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFWhisperForConditionalGeneration for predictions without further training.
10it [01:30,  9.00s/it]


Teacher: avg. time on example: 9005.082321166992
WER on model = 43.47826086956522


In [8]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
    "number_of_clusters": 4,
    "cluster_centroids_init": CentroidInitialization.KMEANS_PLUS_PLUS,
}

clustered_model = cluster_weights(model, **clustering_params)

ValueError: Arguments clone_function and input_tensors are only supported for Sequential models or Functional models. Received model of type 'TFWhisperForConditionalGeneration', with clone_function=<function _cluster_weights.<locals>._add_clustering_wrapper at 0x7ff59c03aee0> and input_tensors=None

In [9]:
# -------------- TF = shit -------------------


from sklearn.cluster import KMeans, DBSCAN


def weight_clustering(model):
    model.to("cpu")
    with torch.no_grad():
        count = 0
        for name, params in model.named_parameters():
            param_shape = list(params.size())
            weights = params.reshape(-1, 1)
            kmeans = KMeans(n_clusters=5, random_state=0).fit(weights)
            cluster_centers = torch.from_numpy(kmeans.cluster_centers_)
            print("Processing for layer ", count)
            count += 1
            # print(type(cluster_centers))
            cluster_list = []
            for i in range(0, len(kmeans.labels_)):
                if kmeans.labels_[i] == 0:
                    cluster_list.append(cluster_centers[0].view(1))
                    # print(cluster_list[i].data_ptr() == cluster_centers[0].data_ptr())
                elif kmeans.labels_[i] == 1:
                    cluster_list.append(cluster_centers[1].view(1))
                    # print(cluster_list[i].data_ptr() == cluster_centers[1].data_ptr())
                elif kmeans.labels_[i] == 2:
                    cluster_list.append(cluster_centers[2].view(1))
                    # print(cluster_list[i].data_ptr() == cluster_centers[2].data_ptr())
                elif kmeans.labels_[i] == 3:
                    cluster_list.append(cluster_centers[3].view(1))
                    # print(cluster_list[i].data_ptr() == cluster_centers[3].data_ptr())
                elif kmeans.labels_[i] == 4:
                    cluster_list.append(cluster_centers[4].view(1))
                    # print(cluster_list[i].data_ptr() == cluster_centers[4].data_ptr())

            reshape_size_tuple = tuple(param_shape)
            cluster_list = torch.tensor(cluster_list)
            cluster_list = cluster_list.reshape(reshape_size_tuple)
            params.data = cluster_list.data
            print(params.data_ptr() == cluster_list.data_ptr())

    return model

In [3]:
processor = WhisperProcessor.from_pretrained("openai/whisper-base", cache_dir=HF_CACHE)
model = WhisperForConditionalGeneration.from_pretrained(
    "openai/whisper-base", cache_dir=HF_CACHE
).to("cpu")

In [23]:
cl_model = weight_clustering(model)

/raid/nanosemantics/vyaznikov/itmo_hw/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Processing for layer  0
True


/raid/nanosemantics/vyaznikov/itmo_hw/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Processing for layer  1
True


/raid/nanosemantics/vyaznikov/itmo_hw/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


...

Processing for layer  242
True


/raid/nanosemantics/vyaznikov/itmo_hw/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Processing for layer  243
True


/raid/nanosemantics/vyaznikov/itmo_hw/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Processing for layer  244
True


In [26]:
def get_model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024**2
    return size_all_mb


size_all_mb = get_model_size(cl_model)
print("model size: {:.3f}MB".format(size_all_mb))

model size: 553.848MB


In [33]:
def map_to_pred(batch, model, processor):
    audio = batch["audio"]
    audio = AudioSegment(audio["bytes"], frame_rate=16000, sample_width=2, channels=1)
    audio = np.array(audio.get_array_of_samples(), dtype=np.int16) / 32768

    input_features = processor.feature_extractor(
        audio, sampling_rate=16000, return_tensors="pt"
    ).input_features
    forced_decoder_ids = processor.get_decoder_prompt_ids(
        language="russian", task="transcribe"
    )

    with torch.no_grad():
        predicted_ids = model.generate(
            input_features.type(torch.float64).to("cuda"),
            forced_decoder_ids=forced_decoder_ids,
        )[0]

    transcription = processor.decode(predicted_ids)
    return processor.tokenizer._normalize(transcription)


def evaluate(processor, model):
    validation_dataset = load_dataset(
        "bond005/sberdevices_golos_10h_crowd",
        "clean",
        split="validation",
        cache_dir="/raid/nanosemantics/vyaznikov/itmo_hw",
    )
    validation_dataset_slice = islice(validation_dataset, NUM_EXAMPLES)

    res = defaultdict(list)

    t = time()
    for el in tqdm(validation_dataset_slice):
        res["reference"].append(processor.tokenizer._normalize(el["transcription"]))
        res["prediction"].append(map_to_pred(el, model, processor))

    t = ((time() - t) / NUM_EXAMPLES) * 1000
    print(f"Teacher: avg. time on example: {t}")

    wer = load("wer")
    print(
        f"WER on model = {100 * wer.compute(references=res['reference'], predictions=res['prediction'])}"
    )

In [34]:
evaluate(processor, cl_model.to("cuda"))

10it [00:46,  4.65s/it]


Teacher: avg. time on example: 4653.657031059265
WER on model = 4880.434782608695


In [12]:
all_weights = torch.cat([w.flatten() for w in model.parameters()])
weights_np = all_weights.cpu().detach().numpy().reshape(-1, 1)
weights_np.shape

(72593920, 1)

In [13]:
cluster_weights = KMeans(n_clusters=16).fit(weights_np)

/raid/nanosemantics/vyaznikov/itmo_hw/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [21]:
def _map_weights_to_idx(module):
    if hasattr(module, "weight") and module.weight is not None:
        weights_np = module.weight.data.cpu().numpy().reshape(-1, 1).astype(np.float32)
        centroids_np = cluster_weights.predict(weights_np).flatten()
        module.weight.data = torch.Tensor(
            centroids_np.reshape(module.weight.data.shape)
        )
        module.weight.data.to(module.weight.data.device)
        module.weight.requires_grad = False
        module.weight.data = module.weight.data.to(torch.int8)

    return module

In [22]:
clustered_model = model.apply(_map_weights_to_idx)

In [24]:
compressed_dict = {
    "centroids": cluster_weights.cluster_centers_.flatten(),
    "quantized_indices": clustered_model.state_dict(),
}
torch.save(compressed_dict, "/raid/nanosemantics/vyaznikov/itmo_hw/itmo_compression_course/hw2/cluster_model.pt")